# Week 2a: coincidence problem

In [1]:
'''
Week 2a coincidence problem
'''
from numpy import random

def acquaintance():
    common = False;
    k = 0
    success = 0
    N = 10000
    population = 200000
    frA = 200
    frB = 200
    
    for i in range(N):
        common = False
        s = []
        for j in range(0, frB): # generate friends of B
            while True:
                k = random.randint(1, population)
                if k not in s:
                    break
            s = s + [k];
            ### BEGIN SOLUTION
            if k <= frA:
                common = True
                break
        if common:
            success = success + 1
        ### END SOLUTION
    print('Probability of common acquaintance = %g' % (success/N))
    
### MAIN ###
acquaintance()

KeyboardInterrupt: 

# Week 6a: central limit theorem

In [ ]:
### IMPORTS
from numpy import random
import matplotlib.pyplot as plot

### Central limit function
def centrallimit():
    n = 10
    N = 300000
    sum = 0.0
#     file = open("data.txt", "w");
    results = []
    for i in range(N):
        sum = 0.0
        for j in range(n):
            if random.random() < 0.5:
                sum = sum + random.binomial(20, 0.1)
            else:
                sum = sum + random.binomial(20, 0.9);
        results = results + [sum]
    return results
#         file.write("\%g " % sum)
#     write(file, "\%g ", sum);
#     file.close()
    
# def results():
    
    
### MAIN ###
results = centrallimit()

In [ ]:
h = plot.hist(results, density = True, bins = 100)

# Week 6a: Confidence intervals for waiting time in single machine station

In [2]:
from PyCh import *
from statistics import mean


# =================================
# # Tote definition
# =================================
@dataclass
class Job:
    entrytime: float = 0
    time_entered_buffer: float = 0

# =================================
# # Generator definition
# =================================        
def Generator(env, sending_channel, ta_distribution):
    while True:
        x = Job()
        x.entrytime = env.now
        send = sending_channel.send(x)
        yield env.select(send)
        delay = ta_distribution()
        yield env.timeout(delay)

# =================================
# # Buffer definition
# =================================
def Buffer(env, receiving_channel, sending_channel):
    xs = [] # list of jobs
    while True:
        receiving = receiving_channel.receive() 
        sending = sending_channel.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            waitingtime.append(env.now - xs[0].time_entered_buffer)
            xs = xs[1:]
        if selected(receiving):
            x = receiving.received_entity
            x.time_entered_buffer = env.now
            xs = xs + [x]

# =================================
# # Machine definition
# =================================
def Machine(env, receiving_channel, sending_channel, tp_distribution ):
    while True:
        receive = receiving_channel.receive()
        x = yield env.select(receive)
        
        delay = tp_distribution()
        yield env.timeout(delay)
        
        send = sending_channel.send(x)
        yield env.select(send)         

# =================================
# # Exit definition
# =================================
def Exit(env, receiving_channel, nJobs):
    total_flowtime = 0.0
    for j in range(nJobs):
        receive = receiving_channel.receive()
        x = yield env.select(receive)
        flowtime.append(env.now - x.entrytime);

# =================================
## GBME Model
# =================================
def GBME():
    env = Environment()

  
    ta_distribution = lambda: random.uniform(0.0, 2.0)
    tp_distribution = lambda: random.uniform(0.0, 1.0)
    nJobs = 1000

    a = Channel(env)
    b = Channel(env)
    c = Channel(env)

    G = env.process(Generator(env, a, ta_distribution))
    B = env.process(Buffer(env, a, b))
    M = env.process(Machine(env, b, c, tp_distribution))
    E = env.process(Exit(env, c, nJobs))

    env.run(until=E)
    
    mean_waitingtime = mean(waitingtime)
    mean_flowtime = mean(flowtime)
    
    return mean_waitingtime, mean_flowtime

# =================================
## Main
# =================================
for i in range(5):
    flowtime = []
    waitingtime = []
    mean_waitingtime, mean_flowtime = GBME()
    print("Mean waiting time in run %d = %g" % (i+1, mean_waitingtime) )
    

PyCh version 0.1 imported succesfully.
 
Mean waiting time in run 1 = 0.150525
Mean waiting time in run 2 = 0.146917
Mean waiting time in run 3 = 0.202253
Mean waiting time in run 4 = 0.173167
Mean waiting time in run 5 = 0.148623


# Week 6b: Serial production lines

In [ ]:
# =================================
# # Exit definition
# =================================
def Exit(env, receiving_channel, nJobs):
    total_flowtime = 0.0
    for j in range(nJobs):
        receive = receiving_channel.receive()
        x = yield env.select(receive)
        flowtime.append(env.now - x.entrytime);
        print(j, 'has exited the system')

##############
env = Environment()

flowtime = []
waitingtime = []

a = [Channel(env) for c in range(3)] # a channel for each tier, each sending totes
b = [Channel(env) for c in range(3)] # a channel for each tier, each sending totes

G  = env.process(Generator(env, a[0], lambda: random.exponential(0.5)))
B1 = env.process(Buffer(   env, a[0], b[0]))
M1 = env.process(Machine(  env, b[0], a[1], lambda: 0.33))
B2 = env.process(Buffer(   env, a[1], b[1]))
M2 = env.process(Machine(  env, b[1], a[2], lambda: random.uniform(0.0,0.8)))
E  = env.process(Exit(     env, a[2], 100000))

env.run()

print("Mean flowtime = %g" % mean(flowtime) )
